# 모델 구조

In [61]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Global State

In [62]:
from langgraph.graph import MessagesState
import pandas as pd
from dataclasses import field


class State(MessagesState):
    
    # 입력된 데이터
    input_id: str = ''  
    input_category: str = ''
    
    # 메타데이터 검색을 통해 확인한 제목과 설명
    subject: str = ''   
    description: str = ''
    
    # LLM이 생성한 검색어 리스트
    query : list[str] = []
    
    # 검색어를 통해 검색한 전체 데이터
    search_df: pd.DataFrame = field(default_factory=pd.DataFrame)

    # 연관성 검색을 통해 확인한 topK 데이터
    relevance_df: pd.DataFrame = field(default_factory=pd.DataFrame)
    
    # 결과
    result_df: pd.DataFrame = field(default_factory=pd.DataFrame)


## 논문 - API 호출용 토큰 생성

In [63]:
import json, base64, requests
from Crypto.Cipher import AES
from datetime import datetime

class AESTestClass:
    def __init__(self, plain_txt, key):
        # iv, block_size 값은 고정
        self.iv = 'jvHJ1EFA0IXBrxxz'
        self.block_size = 16
        self.plain_txt = plain_txt
        self.key = key

    def pad(self):
        # PKCS#7 패딩
        number_of_bytes_to_pad = self.block_size - len(self.plain_txt) % self.block_size
        ascii_str = chr(number_of_bytes_to_pad)
        padding_str = number_of_bytes_to_pad * ascii_str
        return self.plain_txt + padding_str

    def encrypt(self):
        cipher = AES.new(self.key.encode('utf-8'), AES.MODE_CBC, self.iv.encode('utf-8'))
        padded_txt = self.pad()
        encrypted_bytes = cipher.encrypt(padded_txt.encode('utf-8'))
        # URL-safe Base64
        encrypted_str = base64.urlsafe_b64encode(encrypted_bytes).decode('utf-8')
        return encrypted_str
    
def call_access_token(MAC_ADDRESS, API_KEY, CLIENT_ID):
    # 맥주소
    mac = (MAC_ADDRESS or "").strip().strip('"').strip("'").upper().replace(":", "-")
    if not mac:
        raise SystemExit("MAC_ADDRESS가 비어있음")

    # datetime 생성
    dt = datetime.now().strftime('%Y%m%d%H%M%S')

    # JSON 페이로드 생성
    payload = {
        "mac_address": mac,
        "datetime": dt
    }
    plain_json = json.dumps(payload, separators=(',', ':'))

    # AES 암호화 → Base64
    aes = AESTestClass(plain_txt=plain_json, key=API_KEY)
    b64_cipher = aes.encrypt()

    # 인코딩 + 토큰 요청
    endpoint = "https://apigateway.kisti.re.kr/tokenrequest.do"
    params = {
        "accounts": b64_cipher,
        "client_id": CLIENT_ID
    }

    response = requests.get(endpoint, params=params, timeout=10)
    response.raise_for_status()

    data = response.json()
    token = data['access_token'] 
    return token

## 논문 - 함수

In [64]:
import xml.etree.ElementTree as ET
import pandas as pd
import json

def xml_to_df(xml):
    # XML 파싱
    root = ET.fromstring(xml)

    # recordList 찾기
    record_list_element = root.find('recordList')

    # 데이터를 담을 리스트
    records = []

    if record_list_element is not None:
        # 각 record에 대해 반복
        for record_element in record_list_element.findall('record'):
            record_data = {}
            # 각 item에 대해 반복
            for item_element in record_element.findall('item'):
                meta_code = item_element.get('metaCode')
                # CDATA 섹션의 텍스트 추출
                value = item_element.text.strip() if item_element.text else ''
                record_data[meta_code] = value
            records.append(record_data)

    df = pd.DataFrame(records)
    return df

def transform_query(input_query):

    query = {
        "BI": input_query,  # 전체
        # "TI": None,  # 논문명
        # "AU": None,  # 저자
        # "AB": None,  # 초록
        # "KW": None,  # 키워드
        # "PB": None,  # 출판사(발행기관)
        # "SN": None,  # ISSN
        # "BN": None,  # ISBN
        # "PY": None,  # 발행년도
        # "CN": None,  # 문헌번호
        # "DI": None   # DOI
    }

    json_query = json.dumps(query, separators=(',', ':')) 

    return json_query

## 논문 - 제목, 초록 가져오기

In [65]:
import os, requests, xmltodict

def ARTI_browse(state: State):
    
    CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
    ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
    MAC_ADDRESS = os.getenv("MAC_ADDRESS")

    access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

    url = "https://apigateway.kisti.re.kr/openapicall.do"
    params = {
        "client_id": CLIENT_ID,
        "token": access_token,
        "version": 1.0,
        "action": "browse",
        "target": "ARTI",
        "cn": state['input_id'],
        "include": "",
        "exclude": None,
    }
    
    res = requests.get(url, params=params, timeout=20)
    xml = res.text
    dict_data = xmltodict.parse(xml)
    with open("../data/input_data.json", "w", encoding="utf-8") as f:
        json.dump(dict_data, f, ensure_ascii=False, indent=4)

    df = xml_to_df(xml)
    
    subject, description = df['Title'].iloc[0], df['Abstract'].iloc[0]
    
    print('\n[subject]\n', subject)
    print('\n[description]\n', description)
    
    return {'subject': subject, 'description': description}


## 데이터셋 - 제목, 설명 가져오기

In [66]:
import os, requests, json

def DATA_browse(state: State):

    API_KEY = os.getenv("DATAON_META_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_META_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/" + state["input_id"]
    params = {"key": API_KEY}

    res = requests.get(url, params=params, timeout=20)
    data = res.json()

    # json 저장
    with open("../data/input_data.json", "w", encoding="utf-8") as f:
        json.dump(data['records'], f, ensure_ascii=False, indent=4)

    subject, description = data['records']['dataset_title_etc_main'], data['records']['dataset_expl_etc_main']
    print('\n[subject]\n', subject)
    print('\n[description]\n', description)

    return {'subject':subject, 'description' : description} 

## 검색어 생성

In [67]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from typing_extensions import Annotated
from pydantic import Field, BaseModel

# Schema
class QueryResult(BaseModel):
    query: Annotated[
        list[str],
        Field(
            ..., 
            max_length=5, 
            min_length=3,
            description="가장 적절한 검색어들의 리스트, 길이 최소 3개/최대 5개", 
        )
    ]
    
# Prompt
query_template = '''
주어진 제목과 설명을 바탕으로, 가장 연관성이 높은 논문과 데이터셋을 검색하려 합니다.

가장 의미적으로 관련성이 높은 논문과 데이터셋을 검색할 수 있을 쿼리를 만들어 주세요.

[조건]
1. 쿼리는 2~3단어로 구성되어야 합니다.
2. 검색 방식은 쿼리와 정확히 일치하는 내용이 있는 논문이나 데이터를 반환하는 형식입니다.

[Input]
연구 주제: {subject}
연구 설명: {description}

[Output]
가장 적절한 3~5개의 쿼리를 JSON으로 출력해주세요.
'''

query_prompt = PromptTemplate.from_template(query_template)

# Node
def generate_query(state: State):

    prompt = query_prompt.invoke(
        {
            'subject': state['subject'], 
            'description': state['description'],
        }
    )

    # sLLM
    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(QueryResult)
    res = structured_sllm.invoke(prompt)
    query = res.query
    print('\n[query]\n', query)

    return {'query': query}


## 논문 - 쿼리에 따른 검색

In [68]:
import os, requests

def ARTI_search(state: State):
    
    CLIENT_ID = os.getenv("SCIENCEON_CLIENT_ID")
    ARTI_KEY = os.getenv("SCIENCEON_API_KEY")
    MAC_ADDRESS = os.getenv("MAC_ADDRESS")

    access_token = call_access_token(MAC_ADDRESS, ARTI_KEY, CLIENT_ID)

    url = "https://apigateway.kisti.re.kr/openapicall.do"
    df = pd.DataFrame()
    
    for query in state['query']:
        params = {
            "client_id": CLIENT_ID,
            "token": access_token,
            "version": 1.0,
            "action": "search",
            "target": "ARTI",
            "searchQuery": transform_query(query),
            'curPage': 1, # 현재페이지 번호
            'rowCount': 20, # 디스플레이 건수(기본값 10, 최대값 100)
        }

        res = requests.get(url, params=params, timeout=20)
        xml = res.text
        tmp = xml_to_df(xml)
        tmp["query"] = query
        df = pd.concat([df, tmp], ignore_index=True)
        
    df = df.drop_duplicates(subset='CN')
    df.to_csv('../data/search_results_article.csv', index=False, encoding='utf-8')
    print('\n[total article length]\n', len(df))

    cleaned_df = (
        df[
            ['CN', 'Title', 'Abstract', 'Pubyear', 'Author', 'ContentURL', 'query']
        ]
        .rename(
            columns={
                'CN': 'ID',
                'Title': 'title',
                'Abstract': 'description',
                'Pubyear': 'pubyear',
                'Author': 'author',
                'ContentURL': 'URL'
            }
        )
    )

    cleaned_df['category'] = 'article'

    search_df = state.get('search_df')
    
    if search_df is not None and not search_df.empty:
        return {'search_df': pd.concat([search_df, cleaned_df], ignore_index=True)}
    else:
        return {'search_df': cleaned_df}


## 데이터셋 - 쿼리에 따른 검색

In [69]:
import os, requests
import pandas as pd

def DATA_search(state: State):
    
    API_KEY = os.getenv("DATAON_SEARCH_API_KEY")
    assert API_KEY and API_KEY.strip(), "환경변수(DATAON_API_KEY)가 비어있어요!"

    url = "https://dataon.kisti.re.kr/rest/api/search/dataset/"
    df = pd.DataFrame()
    for query in state['query']:
        params = {"key": API_KEY, "query": query, "from": 0, "size": 20}
        # key / CHAR / 필수 / API_KEY
        # query / CHAR / 필수 / 검색키워드
        # from / CHAR / 옵션 / 페이지시작위치
        # size / CHAR / 옵션 / 페이지사이즈

        res = requests.get(url, params=params, timeout=20)
        data = res.json()
        
        if "records" in data:
            tmp = pd.DataFrame(data["records"])
            tmp["query"] = query
            df = pd.concat([df, tmp], ignore_index=True)

    df = df.drop_duplicates(subset='svc_id')
    df.to_csv('../data/search_results_dataset.csv', index=False, encoding='utf-8')
    print('\n[total dataset length]\n', len(df))
    
    cleaned_df = (
        df[
            ['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main','dataset_pub_dt_pc', 'dataset_creator_etc_main', 'dataset_lndgpg', 'query']
        ]
        .rename(
            columns={
                'svc_id': 'ID',
                'dataset_title_etc_main': 'title',
                'dataset_expl_etc_main': 'description',
                'dataset_pub_dt_pc': 'pubyear',
                'dataset_creator_etc_main': 'author',
                'dataset_lndgpg': 'URL',
            }
        )
    )

    cleaned_df['category'] = 'dataset'
    
    search_df = state.get('search_df')
    
    if search_df is not None and not search_df.empty:
        return {'search_df': pd.concat([search_df, cleaned_df], ignore_index=True)}
    else:
        return {'search_df': cleaned_df}


## 연관된 데이터 선정

In [70]:
from typing_extensions import Annotated
from pydantic import Field, BaseModel
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Schema
class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[str],
        Field(
            ..., 
            max_length=5, 
            min_length=3,
            description=(
                "가장 관련성이 높은 데이터의 ID 목록, 길이 최소 3개/최대 5개"
                "각 ID에 대해 relevance(관련도 점수)와 reason(선정 사유)을 동일한 인덱스 위치에 작성합니다. "
            ), 
        )
    ]
    relevance: Annotated[
        list[
            Annotated[
                int, 
                Field(ge=0, le=100, description="0~100 정수")
            ]
        ],
        Field(
            ..., 
            max_length=5, 
            min_length=3,
            description="각 ID에 대응하는 관련도 점수 (0~100 정수)",
        )
    ]
    reason: Annotated[
        list[str],
        Field(
            ..., 
            max_length=5, 
            min_length=3,
            description="각 ID가 선정된 이유를 설명하는 문자열 목록. relevant_id와 인덱스가 일치해야 합니다.",
        )
    ]

# Prompt
relevance_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 제목
- 설명

[목표]
주어진 연구 주제와 가장 관련성 높은 3~5개의 데이터 항목을 선택하세요.
각 항목에 대해 관련도 점수(0~100)와 선정 사유를 함께 작성하세요.


[Input]
연구 주제: {subject}
연구 설명: {description}

[Data]
데이터 목록:
{data}

[Output]
다음 형식의 JSON을 출력하세요:

{{
  "relevant_id": [],
  "relevance": [],
  "reason": []
}}
'''

relevance_prompt = PromptTemplate.from_template(relevance_template)

# Node
def select_relevance_llm(state: State):

    df, subject, description = state['search_df'], state['subject'], state['description']

    prompt = relevance_prompt.invoke(
        {
            'subject': subject, 
            'description': description,
            'data': df[['ID', 'title', 'description']].to_dict(orient="records"),
        }
    )

    sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    structured_sllm = sllm.with_structured_output(IDRelevance)
    res = structured_sllm.invoke(prompt)
        
    relevance_df = pd.DataFrame({
        'ID': res.relevant_id,
        'relevance': res.relevance,
        'reason': res.reason
    })
    
    return {'relevance_df': relevance_df}

In [71]:
# text-embedding-3-small 사용
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from tqdm import tqdm
import pandas as pd

def select_relevance_embedding(state: State):
    
    df, subject, description = state['search_df'], state['subject'], state['description']

        
    df['title'] = df['title'].fillna('')
    df['description'] = df['description'].fillna('')

    MAX_LENGTH = 2000
    df['description'] = df['description'].str.slice(0, MAX_LENGTH)

    texts = (df['title'] + " " +
            df['description']).tolist()

    # 2. 임베딩 객체 생성
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

    # 3. Document 객체로 변환
    docs = [Document(page_content=text, metadata={"ID": row.ID}) 
            for text, row in zip(texts, df.itertuples())]

    batch_size = 500
    stores = []
    for i in tqdm(range(0, len(docs), batch_size)):
        batch = docs[i:i+batch_size]
        store = FAISS.from_documents(batch, embeddings)
        stores.append(store)

    # 부분 벡터스토어 통합
    vectorstore = stores[0]
    for s in stores[1:]:
        vectorstore.merge_from(s)

    # 5. 쿼리 임베딩 생성
    query = subject + " " + description
    query_embedding = embeddings.embed_query(query)

    # 6. 유사 문서 검색
    results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=20)
    
    # 7. 결과 출력
    relevance_df = pd.DataFrame([
        {
            "ID": r.metadata.get("ID"),
            "relevance": score,
        }
        for r, score in results_with_score
    ]).sort_values(by="relevance", ascending=True)

    relevance_df['reason'] = ''
    return {'relevance_df': relevance_df}

## 결과 정리

In [72]:
def summarize_results(state: State):
    
    merged_df = pd.merge(
        state['relevance_df'],
        state['search_df'][['ID','category', 'title', 'description', 'URL']],
        on='ID',
        how='left'
    )

    result_df = merged_df[['category', 'title', 'description', 'relevance', 'reason', 'URL']]
    result_df = result_df.rename(columns={
        'category': '구분',
        'title': '제목',
        'description': '설명',
        'relevance': '점수',
        'reason': '추천 사유',
    })

    return {'result_df': result_df}

# 그래프 구조

In [73]:
def input_router(state):

    if state['input_category'] == 'article':
        return 'article'
    
    if state['input_category'] == 'dataset':
        return 'dataset'

    return 'END'

In [74]:
from langgraph.graph import START, END, StateGraph

def build_graph():
    builder = StateGraph(State)
    
    builder.add_sequence([generate_query, DATA_search, ARTI_search, select_relevance_embedding, summarize_results]) 
    builder.add_node('DATA_browse', DATA_browse)
    builder.add_node('ARTI_browse', ARTI_browse)
    
    builder.add_conditional_edges(
        START,
        input_router,
        {
            'article': 'ARTI_browse',
            'dataset': 'DATA_browse'
        }
    )
    
    builder.add_edge('ARTI_browse', 'generate_query')
    builder.add_edge('DATA_browse', 'generate_query')
    
    builder.add_edge('summarize_results', END)
    
    return builder.compile()

graph = build_graph()

In [75]:
input_id = 'JAKO200411922932805'
input_category = 'article'

# input_id = 'b37f0c9413eeb7c45f6fe31cbe3a41ef'
# input_category = 'dataset'

print('[input_id]\n', input_id)

res = graph.invoke({
    'input_id': input_id,
    'input_category': input_category,
})

display(res['result_df'])

[input_id]
 JAKO200411922932805

[subject]
 한국인의 3차원 무릎관절 구축 및 형상 측정

[description]
 It is necessary to have a model that describes the feature of the knee Joint with a sufficient accuracy. Koreans, however, do not have their own knee joint model to be used in the total knee replacement arthroplasty. They have to use European or American models which do not match Koreans. Three-dimensional visualization techniques are found to be useful in a wide range of medical applications. Three-dimensional imaging studies such as CT(computed tomography) and MRI(magnetic resonance image) provide the primary source of patient-specific data. Three-dimensional knee joint models were constructed by image processing of the CT data of 10 subjects. Using the constructed model, the dimensions of Korean knee joint were measured. And this study proposed a three-dimensional model and data, which can be helpful to develop Korean knee implants and to analyze knee joint movements.

[query]
 ['3D knee model', 'Ko

100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


,구분,제목,설명,점수,추천 사유,URL
0,article,한국인의 3차원 무릎 관절 모델 구축 및 형상 측정,It is necessary to have a model that describes...,0.043304,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
1,article,Determination of representative dimension para...,<P> Knee joint implants developed by western c...,0.495882,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
2,article,Construction of 3D model of knee joint motion ...,<P><B>Abstract</B></P><P>There is a growing de...,0.589866,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
3,article,A method of vivo human knee joint 3D model rec...,Objective: To reconstruct a 3D vivo human knee...,0.607654,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
4,article,Study on 3D Model Reconstruction of Human Knee...,<P>Reconstruction of a 3D model for human knee...,0.629766,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
5,article,Validation of Three-dimensional Reconstruction...,<P>This is the first investigation to validate...,0.652460,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
6,article,"Imaging knee position using MRI, RSA/CT and 3D...",<P><B>Abstract</B></P><P>The purpose of this s...,0.654614,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
7,article,A 3D Printed Mechanical Model of the Knee to D...,"<P>In this article, a novel 3D printed knee mo...",0.656167,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
8,article,A 3D Model Analysis of Knee Prosthetic Motion ...,<P>The object of this study is to introduce co...,0.670475,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
9,article,3D reconstruction of bony elements of the knee...,Two separate themes are presented in this pape...,0.670495,,http://click.ndsl.kr/servlet/OpenAPIDetailView...
